## Run ANTs-CT analyses for T1 Normative Dataset

In [3]:
import flywheel
fw = flywheel.Client()
collection_id = '5eb5081448fe1b1e5792a7a9'
gear = fw.lookup('gears/antsct-fw')

In [3]:
# Here we get all the sessions within the collection.
# We use `get_session` to return the associated analyses
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

done


In [16]:
# Create a short list of sessions for testing.
sessions_short = [
    fw.get('5a1f3f4c3b71e5001dfd54ba'), # TOME_3004
    fw.get('5a1f45833b71e5001bfd5427'), # TOME_3005
    fw.get('5a1f44583b71e50018fd524b'), # TOME_3009
    fw.get('5d7fce5c0f98b7003770ace7'), # PE - C005
    fw.get('5d7fcd7a0f98b7004983593d'), # PE - C004
    fw.get('5d02398aa550c600466fd78f'), # NACC 118317
    fw.get('5d024dcea550c600466fd931'), # NACC 118998
    fw.get('5d017abca550c600486fe47d'), # NACC 106122
    fw.get('5e5d21cc782dc81242f1e8e3'), # PHARMA C14
    fw.get('5e5801b06dea3101ae2a744f'), # CBFLOW C02 hup6
    fw.get('5e5801b06dea3101ae2a7457'), # CBFLOW C01 hup6
    fw.get('5e2070a23a9694007a7709a3'), # ZAP C238
    fw.get('5d9ded21a54d350044c0fc49'), # ZAP C472
    fw.get('5e1f97123a9694007a76060d'), # HUP3TLEGACY 119517
    fw.get('5e27d829048f71005705bed8'), # HUP3TLEGACY 124747
    fw.get('5e32f4796dea3101d72a745b'), # HUP6 125277  
]

### For each session, select appropriate acquisition and run gear on

In [1]:
def run_gear(sessions):
    input_t1s = []
    no_t1_sessions = []
    analysis_ids = []
    for session in sessions:
        ## Get acquisition
        potential_t1s = []
        for acq in session.acquisitions():
            class_t1 = False
            class_struct_intent = False
            class_mprage = False
            if acq.files and len(acq.files) > 0:
                for i in range(0, len(acq.files)):
                    try: 
                        class_t1 = 'T1' in acq.files[i]['classification']['Measurement'] 
                        class_struct_intent = 'Structural' in acq.files[i]['classification']['Intent']
                        class_mprage = 'MPRAGE' in acq.files[i]['classification']['Features']
                    except TypeError as te:
                        continue
                    except KeyError as ke:
                        continue                
                    break                    

            # follow a hierarchy of surest identifiers of the desired T1 acquisition (usually mprage, never setter, etc.)
            is_mprage = class_mprage or "mprage" in acq.label.lower() or 't1w_mpr' in acq.label.lower()                           
            if is_mprage and "setter" not in acq.label:
                potential_t1s.insert(0, acq)
            elif class_t1 and class_struct_intent and "setter" not in acq.label:
                potential_t1s.append(acq)
            else:
                pass

        if len(potential_t1s) == 0:
            sub = fw.get(session.parents['subject'])
            no_t1_sessions.append('{}: {}'.format(sub.label, session.label))
        else:
            # add the highest priority acquisition to the input T1s list
            input_t1_acq = potential_t1s[0]
            input_t1s.append(input_t1_acq)

        for f in input_t1_acq.files:
            if 'nii.gz' in f.name:
                input_file = f
                break

        # Run the gear
        inputs = {'t1_anatomy': input_file}
        config = {'denoise': True, 'num-threads': 0, 'trim-neck': True, 'run-quick': False}
        analysis_id = gear.run(analysis_label='antsct_2021-01-08_WT', config=config, inputs=inputs, destination=session)
        analysis_ids.append(analysis_id)
    return input_t1s, no_t1_sessions, analysis_ids

In [ ]:
# (input_t1s, no_t1_sessions, analysis_ids) = run_gear(sessions)

### Re-do failed runs

In [5]:
# copied from download_ct_maps notebook
failed_session_ids = ['5d71218734e0e3bfaf5542b8','5db3114708e5bb62807be0aa', '5e3c30ca899b20c07d1a77b8', '5e3dad8e899b20c07da919fc']
failed_sessions = []
# fw.get every session object and store in list
for ses_id in failed_session_ids:
    failed_sessions.append(fw.get(ses_id))
print(len(failed_sessions))
(f_input_t1s, f_no_t1_sessions, f_analysis_ids) = run_gear(failed_sessions)

4


### Sanity check the length and contents of the list of T1 acquisitions

In [9]:
print(len(f_input_t1s))

53


In [11]:
print(len(f_no_t1_sessions))
f_no_t1_sessions

0


[]

In [2]:
for t1 in f_input_t1s:
    sub = fw.get(t1.parents['subject'])
    ses = fw.get(t1.parents['session'])
    print('{} | {} | {}'.format(t1.label, sub.label, ses.label))